In [139]:
import cv2
import numpy as np
import struct
import open3d as o3d

In [140]:
rgb_file = 'source/frame_00054_rgb.png'
depth_file = 'source/frame_00054_depth.bin'
pose_file = 'source/frame_00054_pose.txt'

In [141]:
def get_depth_image(file):
    p = 0
    depth_image = []
    points_set = np.zeros((1, 3))
    depth_intrinsic = np.genfromtxt('source/depth.cal', skip_footer=6)
    with open(file, 'rb') as f:
        width = struct.unpack('i', f.read(4))[0]
        height = struct.unpack('i', f.read(4))[0]
        while p < width*height:
            empty_num = struct.unpack('i', f.read(4))[0]
            for i in range(empty_num):
                depth_image.append(0)
            full_num = struct.unpack('i', f.read(4))[0]
            for i in range(full_num):
                depth = struct.unpack('h', f.read(2))[0]
                depth_image.append(depth)
            p += empty_num + full_num
        depth_image = np.reshape(depth_image, (height, width))
        depth_image = depth_image / depth_image.max() * 255
        depth_image = depth_image.astype('uint8')
        # cv2.imshow('depth', depth_image)
        for y in range(0, 480):
            for x in range(0, 640):
                d = depth_image[y][x]
                if d > 0:
                    point_x = d * (float(x) - depth_intrinsic[0][2]) / depth_intrinsic[0][0]
                    point_y = d * (float(y) - depth_intrinsic[1][2]) / depth_intrinsic[1][1]
                    point_z = d
                    points_set = np.concatenate((points_set, np.asarray([point_x, point_y, point_z]).reshape((1, 3))))
        return depth_image, np.delete(points_set, 0, axis=0)

In [142]:
def get_color(points_set):
    rgb_intrinsic = np.genfromtxt('source/rgb.cal', skip_footer=6)
    R_1 = np.genfromtxt('source/rgb.cal', skip_footer=2, skip_header=5)
    R = np.zeros((4, 4))
    R[0:3, 0:3] = R_1
    R[3][3] = 1
    c = np.genfromtxt('source/rgb.cal', skip_header=9, skip_footer=1)
    T = np.eye(4)
    T[0:3, 3] = -c
    PI = rgb_intrinsic.dot(np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])).dot(R).dot(T)
    points_set = np.hstack((points_set, np.ones((points_set.shape[0], 1)))).T
    print(points_set.shape)
    return PI.dot(points_set)

In [143]:
img = cv2.imread(rgb_file)
depth_img, point_set = get_depth_image(depth_file)

In [151]:
def get_pose():
    global depth_img
    global point_set
    f_axis = np.genfromtxt(pose_file, skip_footer=1)
    f_center = np.genfromtxt(pose_file, skip_header=3)
    depth_intrinsic = np.genfromtxt('source/depth.cal', skip_footer=6)
    point_x = int(depth_intrinsic[0][2] + depth_intrinsic[0][0] * f_center[0] / f_center[2])
    point_y = int(depth_intrinsic[1][2] + depth_intrinsic[1][1] * f_center[1] / f_center[2])
    d = depth_img[point_y-5:point_y+5, point_x-5:point_x+5].min() / depth_img.max() * 255
    point_x = d * (float(point_x) - depth_intrinsic[0][2]) / depth_intrinsic[0][0]
    point_y = d * (float(point_y) - depth_intrinsic[1][2]) / depth_intrinsic[1][1]
    translation = 10
    d -= translation
    return f_axis, np.array([point_x, point_y, d])

axis, center = get_pose()
print(point_set[0:4, :])
print(center)


def graw_axis(center, pose):
    mat = np.zeros((20, 3))
    element = np.arange(0, 20)
    mat[:, 0] = element
    mat[:, 1] = element
    mat[:, 2] = element
    pose[:, 2] = - pose[:, 2]
    mat = np.concatenate((mat*pose[0], mat*pose[1], mat*pose[2]))
    color = np.zeros(mat.shape)
    color[0:20, 0] = 255            # Red
    color[20:40, 1] = 255           # Green
    color[40:60, 2] = 255           # Blue
    return center+mat, color

rgb_point_set = get_color(point_set)
print(rgb_point_set.shape)
print(rgb_point_set[:, 0:4])
rgb_point_set[0, :] /= rgb_point_set[2, :]
rgb_point_set[1, :] /= rgb_point_set[2, :]
color_set = []
for i in range(rgb_point_set.shape[1]):
    loc = rgb_point_set[:, i]
    x = int(loc[0])
    y = int(loc[1])
    color_set.append(img[y-460][x-64])
    if i % 10000 == 0:
        print(img[y-480][x-64])
color_set = np.array(color_set) / 255

print(color_set.shape)
print(point_set.shape)
axis_center, axis_color = graw_axis(center, axis)

point_set_1 = np.ones(point_set.shape)
point_set_1[:, 0] = -point_set[:, 1]
point_set_1[:, 1] = point_set[:, 0]
point_set_1[:, 2] = point_set[:, 2]

color_set_1 = np.ones(point_set.shape)
color_set_1[:, 0] = color_set[:, 2]
color_set_1[:, 1] = color_set[:, 1]
color_set_1[:, 2] = color_set[:, 0]

point_cloud = o3d.PointCloud()
point_cloud.points = o3d.Vector3dVector(point_set)
point_cloud.colors = o3d.Vector3dVector(color_set_1)

pose_axis = o3d.PointCloud()
pose_axis.points = o3d.Vector3dVector(axis_center)
pose_axis.colors = o3d.Vector3dVector(axis_color)

o3d.draw_geometries([point_cloud, pose_axis])

214.00000244379044
[[  3.9595982  -50.68285702 228.        ]
 [  5.93939731 -50.68285702 228.        ]
 [  6.33535713 -50.68285702 228.        ]
 [  6.70179363 -50.46056379 227.        ]]
[  2.97317202 -13.00762759 204.00000244]
(4, 71080)
(3, 71080)
[[92889.8941933  93908.8940256  94112.69399206 93975.96135218]
 [30681.42044958 30672.45234006 30670.65871815 30535.0041479 ]
 [  241.78836962   241.77010004   241.76644612   240.75986735]]
[203 197 195]
[138 153 173]
[181 180 175]
[108  82  77]
[127  85  63]
[119  74  52]
[73 59 57]
[85 45 34]
(71080, 3)
(71080, 3)
